# Tutorial session about SNAPPY:
## a Snakemake Phylogeny Pipeline for Scalable HIV-1 Subtyping

## 1. What is SNAPPY?

As the name suggests SNAPPY is a pipeline of tools for automating HIV-1 subtyping. In short, after mapping to the reference genome the given target is added to a previously created phylogenetic tree backbone, then the closest phylogenetic pair, the node most recent ancestors, and the branch support values are inferred. This information is then made available to the user in order to make the subtyping decision as informative as possible.

## 2. Setup

This tutotial was designed to run in unix based systems. So if you are using a windows computer please switch now. You can easly access a unix based systems in one of our group's servers. If you can't or don't know how to do it pelase talk to me before the tutorial session.

First of all pelase download SNAPPY and all the necessary material from the following GitHub repo: https://github.com/PMMAraujo/subtyper_snakemake

You can do i ether by downloading a zip folder (like in the image below) or the usual git comand ( git clone https://github.com/PMMAraujo/subtyper_snakemake)

<img src="tutorial/files_tut/2github.png">

Although we could use SNAPPY without Conda I do very mutch recommend it. Conda is a package manager that makes our life a lot easier (more on conda here: https://conda.io/en/latest/). If you already have Conda installed on the system where you are going to run the tutorial please jump to the last paragrath of the setup(before the notes).
If not please download mini-conda (or the full anaconda if you which) and install it on the designated computer. At the time of writting if you go to https://repo.continuum.io/miniconda/ you can download the most recent build for  Linux (Miniconda2-4.5.12-Linux-x86_64.sh). After you downlaod this script you need to give it premission to run on your systems and then run it (chmodd 777 Miniconda2-4.5.12-Linux-x86_64.sh ; ./Miniconda2-4.5.12-Linux-x86_64.sh). After this conda should be succesefully installed on the machine.

To finalize the setup you need to create and environment with all the necessary software and packages to run SNAPPY. For this pelase go to the directory where you have the downloaded folder (where you uncompressed or git cloned it). When inside the folder pelase run: 'conda-env create -f tut_environment.yaml'. This should take some time, mainly because you are downloading many dependencies. Confirm this process was sucessefull by running: 'conda-env list'. If 'tut_subtype_hiv1' is there the environment is ready. Every time you want to run SNAPPY in a new terminal window (or jupyter-notebook for instance) you need to activate this environment (not create it, just activate it). To do this just type 'conda activate tut_subtype_hiv1'. 
You are now ready to go.

Notes: To deactiavte a conda environment just type 'conda deactivate'. To remove a conda environment just type 'conda remove -n name_env -all'. After the tutorial you can unistall Conda as any other software in youyr system.

## 3. SNAPPY tutorial

### How to follow this tutotial:
Please confirm you are in the designated conda environment ( see setup section).

You can type the comands you see in this document in the terminal and run the tutotial that way. I strongly advise you to copy the jupyter notebook ('01_tutorial.ipynb')from the 'tutorial' folder to the main folder ('subtyper_snakemake') and them run a jupyter instance (type: 'jupyter-notebook') and follow the tutorial from there.

In [ ]:
### Needed import for this tutorial to work

from ete3 import Tree
import pandas as pd

### Whats in this folder:

<img src="tutorial/files_tut/whats.png">

In the 'data' folder their are fiels essential for the usage of SNAPPY, this folder and its content should never be changed. The 'input' folder constains the file that will be the input for SNAPPY. By befault the tutorial file is alreday in this fodler. SNAPPY only acepts as inputs multiple seuqnece files in fasta format. The 'scripts' folder containst the necessary scripts for SNAPPY to run,  this folder and its content should never be changed. The 'tutorial' dolder contains materials for this tutorial. The '01_tutorial.ipynb' file is this tutorial in jupyter notebook format. The 'config.yaml' file has configuration information that the user can change to modulate SNAPPY behaviour, more on that afterwards. 'LICENSE' file corresponds to the software distribution license. 'README.md' is a file with instructions about SNAPPY (we are not following those in this tutorial). 'Snakefile' is the file that has all the rules needed to run Snappy. The 'environment.yaml' and 'tut_environment.yaml' are files used to build conda environment as discussed in the setup section.

Lets fire it up and see what happens.
Note: In this tutorial we run SNAPPY in parallel using 4 cpu cores. This should be fine for most of modern cpus. If you think your machine does not meet this prerequisit please decrease the number delete the cores option ( example: 'snakemake subtype_all') 

In [ ]:
! snakemake --cores 4 subtype_all

As you can see SNAPPY started running printing a lot of information to the screen, let's break it in parts:
    - As any other snakemake pipeline the first thing is to build the DAG of jobs. For that snakemake reads the 'Snakefile' provided and inferes the jobs and order they need to be done;
    - As you can the 2nd thing done was the 'rule split'. Rules can be seen as the tasks that need to be performed. This rule in particular is responsable for the creation of multiple fasta files from a single multiple sequence alignment. The output where wrriten to the folder 'split?;
    - After finishing that task the rule 'map_and_align' starts to be run. This rule is responsble for mappying each of the previously obtained fasta files to the HIV-1 reference genome (HXB2). Then each sequence is trimed to to obtained only the genomic region selected by the use. But we didnt select any genomic region when we ran the previous comand right? Well actualy is very common to provide a 'config.yaml' file in snakemake pipelineswhere certain configuration are typed. In this case you can open this file and see that the genomic region selected was 'PR-RT';
    - If you scroll a bit more you will start seeing the rule 'tree_maker'. As the name indicates this rule is responsable for the creation of a phylogenetic tree where the subtype is going to be inferred for each sequence in the study;
    - If you scroll all the way down you will see the final rule 'subtype_all'. This is the rule responsable for reading each phylogenetic tree created in 'tree_maker' analyse it to find the phylogenetic pair of our target sequence and finaly create the output files designated 'all_subtyped.csv' and 'report_all_subtyped.csv'.
    - Please fell free to explore these two files.

Lets look at the first 10 lines of each of the outputs.

In [ ]:
# pandas (python package) comand to open a csv and show first 10 lines
pd.read_csv('all_subtyped.csv', sep=';').head(n=10) 

In [ ]:
# pandas (python package) comand to open a csv and show first 10 lines
pd.read_csv('report_all_subtyped.csv', sep=';').head(n=10) 

But how did SNAPPY produce this outputs? This was infered from the cfreaqted phylogenetic trees. Lets plot some examples to understand it.

In [ ]:
# using ete3 python package to open the tree
t01 = Tree('trees/GU345241.treefile', format=1)
# using ete3 python package to find the node where our target sequence is
parent01 = t01.search_nodes(name='target')[0].up

# using ete3 python package to open the tree
t02 = Tree('trees/MH062032.treefile', format=1)
# using ete3 python package to find the node where our target sequence is
parent02 = t02.search_nodes(name='target')[0].up

In [ ]:
# plot the nodes where our target sequences are respectively
print(parent01), print(parent02)

As we can see for the previous plots the reuslts for the first and the secound target are extremely diffrent. This is due to the capacity to infer from the phylogeny for the two inputs. Feel free to discuss this results.

Right now would be usefull to look at the 'Snakefile' and see all the previously mentioned rules and how the outputs are buildt from the inputs. Feel free to explore 'Snakefile'.

<img src="tutorial/files_tut/snakefile.png">

Several folders where created with this run, each contain intermidiate files that may be usefull for the user:

<img src="tutorial/files_tut/folders.png">

The reason they where not deleted by default is because they may still be usefull. Snakemake keaps track of the taks it needs to perform by knowing which files it needs to create and which files it needs to perform each task. To better understand this try to run the subtyping pipeline again and seem what happens.

In [ ]:
! snakemake --cores 4 subtype_all

Snakemake says there is nothing to be done! It is true, the outputs ('all_subtyped.csv' and 'report_all_subtyped.csv') for the taks we asked to be performed already exist. But please keep in mind that snakemake only looks for the final outputs, it does not check intermidiate files. Lets remove all the aligned files and see what happens when we run the pipeline.

In [ ]:
! rm -r aligned/ ; ls

In [ ]:
! snakemake --cores 4 subtype_all

Snakemake still syas there is nothing to be done because the outputs ('all_subtyped.csv' and 'report_all_subtyped.csv')  for our tasks are already there. What if we delete them and run the pipeline?

In [ ]:
! rm all_subtyped.csv report_all_subtyped.csv ; ls

In [ ]:
! snakemake --cores 4 subtype_all

Only the rule 'subtype_all' was run. Why? Because snakemake looks from the top down to check what it needs to perform the requested task. In this case What it need to create the outputs ('all_subtyped.csv' and 'report_all_subtyped.csv') are the phylogenetic tree inside the 'trees' folder, since all of them are there the pipeline can produce de desires task. Even tho the aligned files are no longer in the folder snakemake does not need them for the requested task. Lets try to delete some of the trees (5) in the 'trees' folder and run the pipeline again. 

In [ ]:
! ls trees 

In [ ]:
! rm trees/AY901089.treefile trees/GU345241.treefile trees/KX159008.treefile trees/LC311988.treefile trees/JN824399.treefile

In [ ]:
! ls trees

In [ ]:
! snakemake --cores 4 subtype_all

Again nothing to do because it already build the the outputs that we requested. Lets delete those and run the pipeline.

In [ ]:
! rm all_subtyped.csv report_all_subtyped.csv ; ls

In [ ]:
! snakemake --cores 4 subtype_all

As you can see snakemake started by running the rule 'map_and_align' and not the rule 'split'. It does not need to run split because all split outputs are still there ('split' folder). What it needs to do is to align the needed sequences (rule 'align_and_map') to build the deleted phylogenetic trees (rule 'tree_maker') to them build the outputs we requested ( rule 'subtype_all').

Please notice that snakemake did not aligned all te sequnces:

In [ ]:
! ls aligned/

It only aligned 5. Why 5? Because we deleted 5 phylogentic trees ( in the 'trees' folder), therefore the alignment needs to be build for those cases but for the remaining cases the trees are still there and therefore do not need to be constructed. This is very usefull and sves a  lot of computational time.

Althought this pipeline was created  with the final objective of subtyping we can access to each and every of its rules and run it independently. For instance this pipeline is extremely usefull to align HIV-1 sequences. Lets delete the previously made alignments and try this.

In [ ]:
! rm -r aligned/ trees/ split/ report_all_subtyped.csv all_subtyped.csv; ls

In [ ]:
! snakemake --cores 4 align_all

In [ ]:
! ls

Please notice taht this time we did not call the rule 'subtype_all', instead we called 'align_all'. As you can see for the privious output, as expected,the 'trees' folder was not created becasue it is not needed to build the alignments. This time a new output file was created 'all_aligned.fasta', which is specific to the rule 'align_all' which was not previously called.

As previously mentioned we can change the 'config.yaml' file to define which HIV-1 genomic region we want our multiple sequence alignment to contain. Currently SNAPPY is caple to produce MSA for 8 HIV-1 regions as you can see below.

<img src="tutorial/files_tut/regions.png">

Lets save the previously build 'PR-RT' alignment to another file name and build a MSA for almost complete genome ('GAG-POL-ENV'). For that lets change the 'config.yaml' file content to correspond to the next picture.

<img src="tutorial/files_tut/GAG_POL_ENV.png">

In [ ]:
! mv all_aligned.fasta pr-rt_msa.fasta; rm -r aligned/ ; ! snakemake --cores 4 align_all

SNAPPY aligned the input sequencesn but this time produced a ner full length alignment (6910 + postions) as you can see in the figures below. Fell free tu use your favorite multiple sequence visualization software and explore the two aligment created('all_aligned.fasta', 'pr-rt_msa.fasta').

<img src="tutorial/files_tut/compare_alignments.png">

As you can see in the 'Snakefile' SNAPPY has more rules ready to use. One extremely usefull is the rule 'delete_all_outputs' which, as the anme indicataes, deletes all the ouputs created by this pipeline so the user can start from the beguining withou worrying about forgotten files.That and other rules are documented in the README file (NOT!!!). Fell free to read the README and explore the otehr SNAPPY functionalities.